In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

connect = f'sqlite:///FPA_FOD_20170508.sqlite'
engine = create_engine(connect)

In [2]:
def load_data(path='FPA_FOD_20170508.sqlite', nrows=10000):
    connect = f'sqlite:///{path}'
    engine = create_engine(connect)

    if nrows != 'all':
        data = pd.read_sql_query(f"""
            SELECT
                *, 
                datetime(DISCOVERY_DATE) as DIS_DATETIME,
                datetime(CONT_DATE) as CON_DATETIME
            FROM 
                Fires 
            LIMIT {nrows};
            """ , engine)
    else:
        data = pd.read_sql_query(f"""
            SELECT
                *, 
                datetime(DISCOVERY_DATE) as DIS_DATETIME,
                datetime(CONT_DATE) as CON_DATETIME
            FROM 
                Fires 
            """ , engine)
    data['DISC_YM'] = data['DIS_DATETIME'].str.extract(r'(\d{4}-\d{2})')

    return data


all_df = load_data(nrows='all')
df = all_df.copy()
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,DIS_DATETIME,CON_DATETIME,DISC_YM
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,None,None,None,None,None,2005,"2,453,403.50",33,1300,9.00,Miscellaneous,"2,453,403.50",33.00,1730,0.10,A,40.04,-121.01,5.00,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@|\x01\x00\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xfe',2005-02-02 00:00:00,2005-02-02 00:00:00,2005-02
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,13,AAC0,PIGEON,None,None,None,None,None,2004,"2,453,137.50",133,0845,1.00,Lightning,"2,453,137.50",133.00,1530,0.25,A,38.93,-120.40,5.00,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@|\x01\x00\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@\xfe',2004-05-12 00:00:00,2004-05-12 00:00:00,2004-05
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,021,A32W,SLACK,None,None,None,None,None,2004,"2,453,156.50",152,1921,5.00,Debris Burning,"2,453,156.50",152.00,2024,0.10,A,38.98,-120.74,13.00,STATE OR PRIVATE,CA,17,017,El Dorado,"b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@|\x01\x00\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xfe'",2004-05-31 00:00:00,2004-05-31 00:00:00,2004-05
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,6,None,DEER,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1400,0.10,A,38.56,-119.91,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00,2004-06
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,7,None,STEVENOT,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1200,0.10,A,38.56,-119.93,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00,2004-06


# Helper Functions

In [5]:
def plot_fire_cause(cause_counts: pd.DataFrame):

    fig = px.bar(cause_counts, x='STAT_CAUSE_DESCR', y='count',
                hover_data=['count'], color='STAT_CAUSE_DESCR',
                labels={'count':'Total Count', 'STAT_CAUSE_DESCR': 'Causes'}, 
                text_auto='.2s',
                height=500,
                width=800,
                title="Distribution of the fire events")
    return fig


def yearly_summary(year_counts: pd.DataFrame):

    fig = px.bar(year_counts, x='FIPS_NAME', y='count',
                hover_data=['count'], color='count',
                labels={'count':'Total Count', 'FIPS_NAME': 'name'}, 
                text_auto='.2s',
                height=500,
                width=800,
                title="Yearly summary of wildfire")
    return fig

def bar_plot(df: pd.DataFrame,
            x: str,
            y: str,
            labels: dict,
            title: str,
            color=None):

    fig = px.bar(df, x=x, y=y,
                color=color,
                labels=labels, 
                text_auto='.2s',
                height=500,
                width=800,
                title=title)
    return fig

In [37]:
year = 2015

df_filter = df[df["FIRE_YEAR"].isin([year, 2014, 2013]) ]

# county_counts = df_filter['FIPS_NAME'].value_counts()\
#                     .to_frame()\
#                     .reset_index()\
#                     .rename(columns={'FIPS_NAME':"count", "index":'FIPS_NAME'})\
#                     .sort_values(by='count', ascending=False)

county_counts = df_filter[["FIPS_NAME", "FIRE_YEAR", "OBJECTID"]]\
                        .groupby(["FIPS_NAME", "FIRE_YEAR"])\
                        .count()\
                        .rename(columns={'OBJECTID':"count"})\
                        .reset_index()\
                        .sort_values(['count'] , ascending=False)

county_counts.head(20)

,FIPS_NAME,FIRE_YEAR,count
2380,Maricopa,2014,1151
3227,Riverside,2013,741
1063,Dallas,2015,647
1957,Jackson,2015,635
4027,Washington,2014,609
1956,Jackson,2014,607
3228,Riverside,2014,582
3229,Riverside,2015,573
2266,Lincoln,2014,566
2267,Lincoln,2015,557


In [28]:
top_county_fig = bar_plot(county_counts.head(30),
            x='FIPS_NAME',
            y='count',
            color='FIRE_YEAR',
            labels={"count": "Number of wildfires", "FIPS_NAME": "County Name"},
            title="Top Fire-prone Counties")
top_county_fig.show()

In [6]:


top_county_fig.show()

In [159]:
df['DISC_YM'] = df['DIS_DATETIME'].str.extract(r'(\d{4}-\d{2})')
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,DIS_DATETIME,CON_DATETIME,DISC_YM
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,None,None,None,None,None,2005,"2,453,403.50",33,1300,9.00,Miscellaneous,"2,453,403.50",33.00,1730,0.10,A,40.04,-121.01,5.00,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@|\x01\x00\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xfe',2005-02-02 00:00:00,2005-02-02 00:00:00,2005-02
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,13,AAC0,PIGEON,None,None,None,None,None,2004,"2,453,137.50",133,0845,1.00,Lightning,"2,453,137.50",133.00,1530,0.25,A,38.93,-120.40,5.00,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@|\x01\x00\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@\xfe',2004-05-12 00:00:00,2004-05-12 00:00:00,2004-05
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,021,A32W,SLACK,None,None,None,None,None,2004,"2,453,156.50",152,1921,5.00,Debris Burning,"2,453,156.50",152.00,2024,0.10,A,38.98,-120.74,13.00,STATE OR PRIVATE,CA,17,017,El Dorado,"b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@|\x01\x00\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xfe'",2004-05-31 00:00:00,2004-05-31 00:00:00,2004-05
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,6,None,DEER,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1400,0.10,A,38.56,-119.91,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00,2004-06
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,7,None,STEVENOT,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1200,0.10,A,38.56,-119.93,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00,2004-06


In [171]:
# df[df["UnitId"] == "USCAPNF"].head()
df["STAT_CAUSE_DESCR"].value_counts().shape

(13,)

In [196]:
date_counts = df[["DISC_YM", "FIRE_YEAR", "OBJECTID"]]\
                .groupby(["DISC_YM", "FIRE_YEAR"])\
                .count()\
                .rename(columns={'OBJECTID':"count"})\
                .reset_index()
date_counts.head()

,DISC_YM,FIRE_YEAR,count
0,1992-01,1992,3603
1,1992-02,1992,7114
2,1992-03,1992,8706
3,1992-04,1992,7810
4,1992-05,1992,8574


In [202]:
fig2 = go.Figure()
# plot
fig2.add_trace(
                # go.Scatter(x=date_counts['DISC_YM'], y=date_counts['count'])
                go.Bar(x=date_counts['DISC_YM'], y=date_counts['count'],colo )
                )

# Set labels and title
fig2.update_layout({"title": "Trend of Wildfire over time",
                   "yaxis": {"title":"Number of Wildfires"},
                   "xaxis": {"title":"Date(Month-Year)"}})

# Add range slider and buttons
# fig2.update_traces(texttemplate = "%{x} | %{y}")
# fig2.update_traces(textposition = "bottom center")
# fig2.update_traces(textposition = "top center")
fig2.update_layout(xaxis=dict(rangeslider=dict(visible=True), type="date"))
# st.plotly_chart(fig2, use_container_width=True)

In [200]:
year_counts = df[["FIRE_YEAR", "OBJECTID"]]\
                .groupby(["FIRE_YEAR"])\
                .count()\
                .rename(columns={'OBJECTID':"count"})\
                .reset_index()
year_counts.head()

,FIRE_YEAR,count
0,1992,67975
1,1993,61989
2,1994,75955
3,1995,71472
4,1996,75574


In [208]:
fig = px.bar(year_counts, y='count', x='FIRE_YEAR', 
            text_auto='.2s',
            labels={'count':'Total Count', 'FIRE_YEAR': 'Year'}, 
            title="Yearly summary of wildfire")
fig.show()

In [186]:
def subdata(df):
    cause_count = df["STAT_CAUSE_DESCR"].value_counts()\
                        .to_frame()\
                        .reset_index()\
                        .rename(columns={'STAT_CAUSE_DESCR':"count", "index":'STAT_CAUSE_DESCR'})\
                        .sort_values(by='STAT_CAUSE_DESCR')

    date_counts = df[["DISC_YM", "FIRE_YEAR"]]\
                        .groupby(["DISC_YM", "FIRE_YEAR"])\
                        .count()\
                        .rename(columns={'DISC_YM':"count"})\
                        .reset_index()
                    
    county_counts = df['FIPS_NAME'].value_counts()\
                        .to_frame()\
                        .reset_index()\
                        .rename(columns={'FIPS_NAME':"count", "index":'FIPS_NAME'})\
                        .sort_values(by='count', ascending=False)

    return cause_count, date_counts, county_counts

cause_count, date_counts, county_counts = subdata(df)
cause_count.head()

,STAT_CAUSE_DESCR,count
2,Arson,281455
6,Campfire,76139
7,Children,61167
0,Debris Burning,429028
5,Equipment Use,147612


In [30]:
cause_count = df[["STAT_CAUSE_DESCR", "FIRE_YEAR", "OBJECTID"]]\
                    .groupby(["STAT_CAUSE_DESCR", "FIRE_YEAR"])\
                    .count()\
                    .rename(columns={'OBJECTID':"count"})\
                    .reset_index()


# cause_count = df["STAT_CAUSE_DESCR"].value_counts()\
#                     .to_frame()\
#                     .reset_index()\
#                     .rename(columns={'STAT_CAUSE_DESCR':"count", "index":'STAT_CAUSE_DESCR'})\
#                     .sort_values(by='STAT_CAUSE_DESCR')


cause_count.head()

,STAT_CAUSE_DESCR,FIRE_YEAR,count
0,Arson,1992,11162
1,Arson,1993,10945
2,Arson,1994,10080
3,Arson,1995,12648
4,Arson,1996,10206


In [35]:
cause_count["FIRE_YEAR"] = cause_count["FIRE_YEAR"].astype(str)
fig = px.bar(cause_count, x='STAT_CAUSE_DESCR', y='count',
                color='FIRE_YEAR',
                labels={'count':'Total Count', 'STAT_CAUSE_DESCR': 'Causes'}, 
                text_auto='.2s',
                height=800,
                title="Distribution of the fire events")
fig.show()

In [195]:

def plot_fire_cause(cause_count):
    fig = px.bar(cause_count, x='STAT_CAUSE_DESCR', y='count',
                hover_data=['count'], color='STAT_CAUSE_DESCR',
                labels={'count':'Total Count', 'STAT_CAUSE_DESCR': 'Causes'}, 
                text_auto='.2s',
                height=800,
                title="Distribution of the fire events")
fig.show()

In [ ]:
def subdata(df):
    cause_count = df["STAT_CAUSE_DESCR"].value_counts()\
                        .to_frame()\
                        .reset_index()\
                        .rename(columns={'STAT_CAUSE_DESCR':"count", "index":'STAT_CAUSE_DESCR'})\
                        .sort_values(by='count', ascending=False)

    date_counts = df[["DISC_YM", "FIRE_YEAR"]]\
                        .groupby(["DISC_YM", "FIRE_YEAR"])\
                        .count()\
                        .rename(columns={'DISC_YM':"count"})\
                        .reset_index()
                    
    county_counts = df['FIPS_NAME'].value_counts()\
                        .to_frame()\
                        .reset_index()\
                        .rename(columns={'FIPS_NAME':"count", "index":'FIPS_NAME'})\
                        .sort_values(by='count', ascending=False)

    return cause_count, date_counts, county_counts

cause_count, date_counts, county_counts = subdata(df)
cause_count.head()

In [147]:
def view_county(county_counts, tail=False, count=50):

    if tail:
        title = "Least Fire-prone Counties"
        slice_county = county_counts.tail(count)
    else:
        title = "Top Fire-prone Counties"
        slice_county = county_counts.head(count)
    fig = go.Figure()

    fig.add_trace(
                    go.Bar(
                        x=slice_county['FIPS_NAME'], 
                        y=slice_county['count'], 
                        texttemplate = "%{y}")
                    )

    fig.update_layout({
                        "title": title,
                        "yaxis": {"title":"Number of Wildfires"},
                        "xaxis": {"title":"County Names"}
                    },
                    xaxis=dict(rangeslider=dict(visible=True)) # add slider
                    )


    return fig



In [148]:
fig  = view_county(county_counts, count=50)
fig.show()

In [128]:
# plot figure
count = 50
slice_county = county_counts.head(count)
fig2 = go.Figure()
# plot
fig2.add_trace(
                go.Bar(x=slice_county['FIPS_NAME'], y=slice_county['count'])
                )
# Set labels and title
fig2.update_layout({
                    "title": "Fire-prone Counties",
                    "yaxis": {"title":"Number of Wildfires"},
                    "xaxis": {"title":"County Names"}
                   })

fig2.update_traces(texttemplate = "%{y}")
# Add range slider and buttons
fig2.update_layout(
                    xaxis=dict(rangeslider=dict(visible=True))
                   )

fig2.show()  

In [46]:
df["STAT_CAUSE_DESCR"].value_counts().index

Index(['Debris Burning', 'Miscellaneous', 'Arson', 'Lightning', 'Missing/Undefined', 'Equipment Use', 'Campfire', 'Children', 'Smoking', 'Railroad', 'Powerline', 'Fireworks', 'Structure'], dtype='object')

In [40]:
month_year_counts = df[["DISC_YM", "STAT_CAUSE_DESCR", "OBJECTID"]]\
                        .groupby(["DISC_YM", "STAT_CAUSE_DESCR"])\
                        .count()\
                        .rename(columns={'OBJECTID':"count"})\
                        .reset_index()

month_year_counts.head()

,DISC_YM,STAT_CAUSE_DESCR,count
0,1992-01,Arson,763
1,1992-01,Campfire,55
2,1992-01,Children,159
3,1992-01,Debris Burning,893
4,1992-01,Equipment Use,301


In [43]:
fig = px.line(month_year_counts, x="DISC_YM", y="count", color="STAT_CAUSE_DESCR", labels={'count':'Total Count', 'STAT_CAUSE_DESCR': 'Causes'}, )
fig.show()

In [267]:
# month_year_counts["FIRE_YEAR"] = month_year_counts["FIRE_YEAR"].astype(str)
fig = px.bar(month_year_counts, x="DISC_YM", y="count", color='FIRE_YEAR')
fig.add_scatter( x=month_year_counts["DISC_YM"], y=month_year_counts["count"], marker=dict(color="blue")) 
fig.show()

In [222]:
month_year_counts.head(100)

,DISC_YM,FIRE_YEAR,count
0,1992-01,1992,3603
1,1992-02,1992,7114
2,1992-03,1992,8706
3,1992-04,1992,7810
4,1992-05,1992,8574
5,1992-06,1992,7307
6,1992-07,1992,7618
7,1992-08,1992,7478
8,1992-09,1992,3717
9,1992-10,1992,3579


In [130]:
# plot figure
count = 100
slice_county = county_counts.tail(count)
fig2 = go.Figure()
# plot
fig2.add_trace(
                go.Bar(x=slice_county['FIPS_NAME'], y=slice_county['count'])
                )
# Set labels and title
fig2.update_layout({
                    "title": "Fire-prone Counties",
                    "yaxis": {"title":"Number of Wildfires"},
                    "xaxis": {"title":"County Names"}
                   })

fig2.update_traces(texttemplate = "%{y}")
# Add range slider and buttons
fig2.update_layout(
                    xaxis=dict(rangeslider=dict(visible=True))
                   )

fig2.show()  

In [217]:
d = df[["STATE", "FIRE_YEAR", "OBJECTID"]]\
                        .groupby(["STATE", "FIRE_YEAR"])\
                        .count()\
                        .rename(columns={'OBJECTID':"count"})\
                        .reset_index()
                        
d["FIRE_YEAR"] = d["FIRE_YEAR"].astype(str)
d.head()

,STATE,FIRE_YEAR,count
0,AK,1992,481
1,AK,1993,861
2,AK,1994,640
3,AK,1995,418
4,AK,1996,718


In [271]:
fig = px.bar(d, x="STATE", y="count", )
# fig.add_scatter( x=d["STATE"], y=d["count"], marker=dict(color="blue")) 
fig.show()

In [ ]:
fig = px.bar(d, x='STATE', y='count',
                hover_data=['count'], color='FIRE_YEAR',
                labels={'count':'Total Count', 'STATE': 'Causes'}, 
                
                height=800,
                width=1000,
                title="State Distribution")
fig.show()

In [ ]:
# df[df["UnitId"] == "USCAPNF"].head() 
df["STATE"].value_counts()

CA    189550
GA    168867
TX    142021
NC    111277
FL     90261
SC     81315
NY     80870
MS     79230
AZ     71586
AL     66570
OR     61088
MN     44769
OK     43239
MT     40767
NM     37478
ID     36698
CO     34157
WA     33513
WI     31861
AR     31663
TN     31154
SD     30963
UT     30725
LA     30013
KY     27089
NJ     25949
PR     22081
WV     21967
VA     21833
MO     17953
NV     16956
ND     15201
WY     14166
ME     13150
AK     12843
MI     10502
HI      9895
PA      8712
NE      7973
KS      7673
CT      4976
IA      4134
MD      3622
OH      3479
MA      2626
NH      2452
IL      2327
IN      2098
RI       480
VT       456
DE       171
DC        66
Name: STATE, dtype: int64

In [ ]:
d = df[["DISC_YM", "FIRE_YEAR", "OBJECTID"]]\
                .groupby(["DISC_YM", "FIRE_YEAR"])\
                .count()\
                .rename(columns={'OBJECTID':"count"})\
                .reset_index()
d.head()

,DISC_YM,FIRE_YEAR,count
0,1992-01,1992,3603
1,1992-02,1992,7114
2,1992-03,1992,8706
3,1992-04,1992,7810
4,1992-05,1992,8574


# All table in DB

In [ ]:
all_table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", engine)
all_table

,name
0,spatial_ref_sys
1,spatialite_history
2,sqlite_sequence
3,geometry_columns
4,spatial_ref_sys_aux
5,views_geometry_columns
6,virts_geometry_columns
7,geometry_columns_statistics
8,views_geometry_columns_statistics
9,virts_geometry_columns_statistics


In [ ]:
f_df = pd.read_sql_query("""
        SELECT
            *, 
            datetime(DISCOVERY_DATE) as DIS_DATETIME,
            datetime(CONT_DATE) as CON_DATETIME
        FROM 
            Fires 
        LIMIT 10000;
        """ , engine)
f_df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,DIS_DATETIME,CON_DATETIME
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,None,None,None,None,None,2005,"2,453,403.50",33,1300,9.00,Miscellaneous,"2,453,403.50",33.00,1730,0.10,A,40.04,-121.01,5.00,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@|\x01\x00\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xfe',2005-02-02 00:00:00,2005-02-02 00:00:00
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,13,AAC0,PIGEON,None,None,None,None,None,2004,"2,453,137.50",133,0845,1.00,Lightning,"2,453,137.50",133.00,1530,0.25,A,38.93,-120.40,5.00,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@|\x01\x00\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@\xfe',2004-05-12 00:00:00,2004-05-12 00:00:00
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,021,A32W,SLACK,None,None,None,None,None,2004,"2,453,156.50",152,1921,5.00,Debris Burning,"2,453,156.50",152.00,2024,0.10,A,38.98,-120.74,13.00,STATE OR PRIVATE,CA,17,017,El Dorado,"b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@|\x01\x00\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xfe'",2004-05-31 00:00:00,2004-05-31 00:00:00
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,6,None,DEER,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1400,0.10,A,38.56,-119.91,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,7,None,STEVENOT,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1200,0.10,A,38.56,-119.93,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00


In [ ]:
# f_df["NWCG_REPORTING_UNIT_ID"].value_counts()
f_df["COUNTY"].value_counts()
# f_df[f_df["NWCG_REPORTING_UNIT_ID"] == "USAKCAN"].head()


5      824
3      441
39     369
7      360
17     339
53     324
49     318
37     314
13     256
23     234
35     202
1      185
25     182
63     176
43     169
19     148
51     143
81     140
47     130
59     129
15     120
11     110
33     103
85      93
65      91
31      91
27      91
111     88
93      88
83      88
21      87
61      85
55      79
29      78
103     78
69      75
75      68
41      66
9       65
147     65
67      62
123     62
89      57
57      51
131     47
109     43
105     40
91      39
113     37
129     35
6       34
79      33
137     32
203     30
77      30
73      28
153     27
241     26
181     25
97      24
221     23
45      22
179     21
149     20
87      18
71      18
117     18
161     17
213     17
115     14
110     13
125     12
165     12
99      12
28      11
215     11
175      9
130      8
107      8
187      7
220      7
235      7
101      7
291      6
223      5
169      5
280      5
237      5
127      5
151      5
510      5

In [ ]:
df = pd.read_sql('SELECT * FROM NWCG_UnitIDActive_20170109', engine)
print(df.shape)
df.head()

(5867, 13)


,OBJECTID,UnitId,GeographicArea,Gacc,WildlandRole,UnitType,Department,Agency,Parent,Country,State,Code,Name
0,1,USAKCAN,AK,USAKACC,Resource Provider Only,Non-Government,NG,ANC,None,US,AK,CAN,Calista Regional Native Corporation
1,2,USAKCEKX,AK,USAKACC,Incident Host Geographic,US County/Local,AK,C&L,None,US,AK,CEKX,Central Emergency Services
2,3,USAKCFFX,AK,USAKACC,Incident Host Geographic,US County/Local,AK,C&L,None,US,AK,CFFX,City Fairbanks Fire Department
3,4,USAKCFMX,AK,USAKACC,Incident Host Geographic,US County/Local,AK,C&L,None,US,AK,CFMX,Central Mat-Su Fire Department
4,5,USAKCFQ,AK,USAKACC,Incident Host Geographic,US Federal,DOD,USAF,None,US,AK,CFQ,Clear Air Force Station


In [ ]:
map_data = pd.DataFrame(
    np.random.randn(1000, 2) / [50, 50] + [37.76, -122.4],
    columns=['lat', 'lon'])

map_data.head()

,lat,lon
0,37.74,-122.38
1,37.77,-122.38
2,37.75,-122.40
3,37.74,-122.40
4,37.72,-122.39


In [ ]:
# df[df["UnitId"] == "USCAPNF"].head()
df["State"].value_counts().shape

KeyError: 'State'

In [ ]:
spatial_ref_sys = pd.read_sql('SELECT * FROM spatial_ref_sys', engine)
ss_df = spatial_ref_sys.copy()
print(spatial_ref_sys.shape)
spatial_ref_sys.head()

(4924, 6)


,srid,auth_name,auth_srid,ref_sys_name,proj4text,srtext
0,-1,NONE,-1,Undefined - Cartesian,,Undefined
1,0,NONE,0,Undefined - Geographic Long/Lat,,Undefined
2,2000,epsg,2000,Anguilla 1957 / British West Indies Grid,+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +y_0=0 +ellps=clrk80 +units=m +no_defs,"PROJCS[""Anguilla 1957 / British West Indies Grid"",GEOGCS[""Anguilla 1957"",DATUM[""Anguilla_1957"",SPHEROID[""Clarke 1880 (RGS)"",6378249.145,293.465,AUTHORITY[""EPSG"",""7012""]],AUTHORITY[""EPSG"",""6600""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AXIS[""Latitude"",NORTH],AXIS[""Longitude"",EAST],AUTHORITY[""EPSG"",""4600""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-62],PARAMETER[""scale_factor"",0.9995],PARAMETER[""false_easting"",400000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""2000""]]"
3,2001,epsg,2001,Antigua 1943 / British West Indies Grid,"+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +y_0=0 +ellps=clrk80 +towgs84=-255,-15,71,0,0,0,0 +units=m +no_defs","PROJCS[""Antigua 1943 / British West Indies Grid"",GEOGCS[""Antigua 1943"",DATUM[""Antigua_1943"",SPHEROID[""Clarke 1880 (RGS)"",6378249.145,293.465,AUTHORITY[""EPSG"",""7012""]],TOWGS84[-255,-15,71,0,0,0,0],AUTHORITY[""EPSG"",""6601""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AXIS[""Latitude"",NORTH],AXIS[""Longitude"",EAST],AUTHORITY[""EPSG"",""4601""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-62],PARAMETER[""scale_factor"",0.9995],PARAMETER[""false_easting"",400000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""2001""]]"
4,2002,epsg,2002,Dominica 1945 / British West Indies Grid,"+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +y_0=0 +ellps=clrk80 +towgs84=725,685,536,0,0,0,0 +units=m +no_defs","PROJCS[""Dominica 1945 / British West Indies Grid"",GEOGCS[""Dominica 1945"",DATUM[""Dominica_1945"",SPHEROID[""Clarke 1880 (RGS)"",6378249.145,293.465,AUTHORITY[""EPSG"",""7012""]],TOWGS84[725,685,536,0,0,0,0],AUTHORITY[""EPSG"",""6602""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AXIS[""Latitude"",NORTH],AXIS[""Longitude"",EAST],AUTHORITY[""EPSG"",""4602""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-62],PARAMETER[""scale_factor"",0.9995],PARAMETER[""false_easting"",400000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""2002""]]"


In [ ]:
ss_df = spatial_ref_sys.iloc[2:]
ss_df.head()

,srid,auth_name,auth_srid,ref_sys_name,proj4text,srtext
2,2000,epsg,2000,Anguilla 1957 / British West Indies Grid,+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +y_0=0 +ellps=clrk80 +units=m +no_defs,"PROJCS[""Anguilla 1957 / British West Indies Grid"",GEOGCS[""Anguilla 1957"",DATUM[""Anguilla_1957"",SPHEROID[""Clarke 1880 (RGS)"",6378249.145,293.465,AUTHORITY[""EPSG"",""7012""]],AUTHORITY[""EPSG"",""6600""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AXIS[""Latitude"",NORTH],AXIS[""Longitude"",EAST],AUTHORITY[""EPSG"",""4600""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-62],PARAMETER[""scale_factor"",0.9995],PARAMETER[""false_easting"",400000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""2000""]]"
3,2001,epsg,2001,Antigua 1943 / British West Indies Grid,"+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +y_0=0 +ellps=clrk80 +towgs84=-255,-15,71,0,0,0,0 +units=m +no_defs","PROJCS[""Antigua 1943 / British West Indies Grid"",GEOGCS[""Antigua 1943"",DATUM[""Antigua_1943"",SPHEROID[""Clarke 1880 (RGS)"",6378249.145,293.465,AUTHORITY[""EPSG"",""7012""]],TOWGS84[-255,-15,71,0,0,0,0],AUTHORITY[""EPSG"",""6601""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AXIS[""Latitude"",NORTH],AXIS[""Longitude"",EAST],AUTHORITY[""EPSG"",""4601""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-62],PARAMETER[""scale_factor"",0.9995],PARAMETER[""false_easting"",400000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""2001""]]"
4,2002,epsg,2002,Dominica 1945 / British West Indies Grid,"+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +y_0=0 +ellps=clrk80 +towgs84=725,685,536,0,0,0,0 +units=m +no_defs","PROJCS[""Dominica 1945 / British West Indies Grid"",GEOGCS[""Dominica 1945"",DATUM[""Dominica_1945"",SPHEROID[""Clarke 1880 (RGS)"",6378249.145,293.465,AUTHORITY[""EPSG"",""7012""]],TOWGS84[725,685,536,0,0,0,0],AUTHORITY[""EPSG"",""6602""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AXIS[""Latitude"",NORTH],AXIS[""Longitude"",EAST],AUTHORITY[""EPSG"",""4602""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-62],PARAMETER[""scale_factor"",0.9995],PARAMETER[""false_easting"",400000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""2002""]]"
5,2003,epsg,2003,Grenada 1953 / British West Indies Grid,"+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +y_0=0 +ellps=clrk80 +towgs84=72,213.7,93,0,0,0,0 +units=m +no_defs","PROJCS[""Grenada 1953 / British West Indies Grid"",GEOGCS[""Grenada 1953"",DATUM[""Grenada_1953"",SPHEROID[""Clarke 1880 (RGS)"",6378249.145,293.465,AUTHORITY[""EPSG"",""7012""]],TOWGS84[72,213.7,93,0,0,0,0],AUTHORITY[""EPSG"",""6603""]],PRIMEM[""Greenwich"",0,AUTHORITY[""EPSG"",""8901""]],UNIT[""degree"",0.0174532925199433,AUTHORITY[""EPSG"",""9122""]],AXIS[""Latitude"",NORTH],AXIS[""Longitude"",EAST],AUTHORITY[""EPSG"",""4603""]],PROJECTION[""Transverse_Mercator""],PARAMETER[""latitude_of_origin"",0],PARAMETER[""central_meridian"",-62],PARAMETER[""scale_factor"",0.9995],PARAMETER[""false_easting"",400000],PARAMETER[""false_northing"",0],UNIT[""metre"",1,AUTHORITY[""EPSG"",""9001""]],AXIS[""Easting"",EAST],AXIS[""Northing"",NORTH],AUTHORITY[""EPSG"",""2003""]]"
6,2004,epsg,2004,Montserrat 1958 / British West Indies Grid,"+proj=tmerc +lat_0=0 +lon_0=-62 +k=0.9995000000000001 +x_0=400000 +

In [ ]:
ss_df.iloc[0, 5]

'PROJCS["Anguilla 1957 / British West Indies Grid",GEOGCS["Anguilla 1957",DATUM["Anguilla_1957",SPHEROID["Clarke 1880 (RGS)",6378249.145,293.465,AUTHORITY["EPSG","7012"]],AUTHORITY["EPSG","6600"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4600"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-62],PARAMETER["scale_factor",0.9995],PARAMETER["false_easting",400000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2000"]]'

In [ ]:
spatial_ref_sys_aux = pd.read_sql('SELECT * FROM spatial_ref_sys_aux', engine)
sa_df = spatial_ref_sys_aux.copy()
print(sa_df.shape)
sa_df.head()

(4873, 12)


,srid,is_geographic,has_flipped_axes,spheroid,prime_meridian,datum,projection,unit,axis_1_name,axis_1_orientation,axis_2_name,axis_2_orientation
0,2000,0,0,Clarke 1880 (RGS),Greenwich,Anguilla_1957,Transverse_Mercator,metre,Easting,East,Northing,North
1,2001,0,0,Clarke 1880 (RGS),Greenwich,Antigua_1943,Transverse_Mercator,metre,Easting,East,Northing,North
2,2002,0,0,Clarke 1880 (RGS),Greenwich,Dominica_1945,Transverse_Mercator,metre,Easting,East,Northing,North
3,2003,0,0,Clarke 1880 (RGS),Greenwich,Grenada_1953,Transverse_Mercator,metre,Easting,East,Northing,North
4,2004,0,0,Clarke 1880 (RGS),Greenwich,Montserrat_1958,Transverse_Mercator,metre,Easting,East,Northing,North


In [ ]:
sa_df.head()

,srid,is_geographic,has_flipped_axes,spheroid,prime_meridian,datum,projection,unit,axis_1_name,axis_1_orientation,axis_2_name,axis_2_orientation
0,2000,0,0,Clarke 1880 (RGS),Greenwich,Anguilla_1957,Transverse_Mercator,metre,Easting,East,Northing,North
1,2001,0,0,Clarke 1880 (RGS),Greenwich,Antigua_1943,Transverse_Mercator,metre,Easting,East,Northing,North
2,2002,0,0,Clarke 1880 (RGS),Greenwich,Dominica_1945,Transverse_Mercator,metre,Easting,East,Northing,North
3,2003,0,0,Clarke 1880 (RGS),Greenwich,Grenada_1953,Transverse_Mercator,metre,Easting,East,Northing,North
4,2004,0,0,Clarke 1880 (RGS),Greenwich,Montserrat_1958,Transverse_Mercator,metre,Easting,East,Northing,North


In [ ]:
ss_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4922 entries, 2 to 4923
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   srid          4922 non-null   int64 
 1   auth_name     4922 non-null   object
 2   auth_srid     4922 non-null   int64 
 3   ref_sys_name  4922 non-null   object
 4   proj4text     4922 non-null   object
 5   srtext        4922 non-null   object
dtypes: int64(2), object(4)
memory usage: 230.8+ KB


In [ ]:
spatialite_history = pd.read_sql('SELECT * FROM NWCGUnitIDActive20170109', engine)
sh_df = spatialite_history.copy()
print(sh_df.shape)
sh_df.head()

OperationalError: (sqlite3.OperationalError) no such table: NWCGUnitIDActive20170109
[SQL: SELECT * FROM NWCGUnitIDActive20170109]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
f_df = pd.read_sql_query("""
        SELECT
            *, 
            datetime(DISCOVERY_DATE) as DIS_DATETIME,
            datetime(CONT_DATE) as CON_DATETIME
        FROM 
            Fires 
        LIMIT 10000;
        """ , engine)
f_df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,DIS_DATETIME,CON_DATETIME
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,None,None,None,None,None,2005,"2,453,403.50",33,1300,9.00,Miscellaneous,"2,453,403.50",33.00,1730,0.10,A,40.04,-121.01,5.00,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@|\x01\x00\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xfe',2005-02-02 00:00:00,2005-02-02 00:00:00
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,13,AAC0,PIGEON,None,None,None,None,None,2004,"2,453,137.50",133,0845,1.00,Lightning,"2,453,137.50",133.00,1530,0.25,A,38.93,-120.40,5.00,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@|\x01\x00\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@\xfe',2004-05-12 00:00:00,2004-05-12 00:00:00
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,021,A32W,SLACK,None,None,None,None,None,2004,"2,453,156.50",152,1921,5.00,Debris Burning,"2,453,156.50",152.00,2024,0.10,A,38.98,-120.74,13.00,STATE OR PRIVATE,CA,17,017,El Dorado,"b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@|\x01\x00\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xfe'",2004-05-31 00:00:00,2004-05-31 00:00:00
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,6,None,DEER,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1400,0.10,A,38.56,-119.91,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,7,None,STEVENOT,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1200,0.10,A,38.56,-119.93,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@\xfe',2004-06-28 00:00:00,2004-07-03 00:00:00


In [ ]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   OBJECTID                    1000 non-null   int64  
 1   FOD_ID                      1000 non-null   int64  
 2   FPA_ID                      1000 non-null   object 
 3   SOURCE_SYSTEM_TYPE          1000 non-null   object 
 4   SOURCE_SYSTEM               1000 non-null   object 
 5   NWCG_REPORTING_AGENCY       1000 non-null   object 
 6   NWCG_REPORTING_UNIT_ID      1000 non-null   object 
 7   NWCG_REPORTING_UNIT_NAME    1000 non-null   object 
 8   SOURCE_REPORTING_UNIT       1000 non-null   object 
 9   SOURCE_REPORTING_UNIT_NAME  1000 non-null   object 
 10  LOCAL_FIRE_REPORT_ID        1000 non-null   object 
 11  LOCAL_INCIDENT_ID           999 non-null    object 
 12  FIRE_CODE                   987 non-null    object 
 13  FIRE_NAME                   1000 n

In [ ]:
from datetime import datetime as dt

def convert_ts(ts):
    """
    Parameter:
            The 'ts' parameter is a millisecond value from the log_data
    Function:
            Convert the timestamp in millisecond to the required format i.e
            hour, day, week of year, month, year and weekday.
    """

    t = dt.fromtimestamp(ts)
    try:
        hour = t.hour
        day =  t.day
        week_of_year = t.isocalendar()[1]
        month = t.month
        year = t.year
        weekday =  t.weekday()

        data = [ts, hour, day, week_of_year, month, year, weekday]

    except Exception as e:
        print(e)

    return data

f_df['date'] = f_df['DISCOVERY_DATE'].apply(convert_ts)


In [ ]:
t = 2453403.50

from datetime import datetime as dt
from datetime import timezone
dt.fromtimestamp(t, timezone.utc)

dt.fromisoformat(t)

TypeError: fromisoformat: argument must be str

In [ ]:
f_df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,date
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,None,None,None,None,None,2005,"2,453,403.50",33,1300,9.00,Miscellaneous,"2,453,403.50",33.00,1730,0.10,A,40.04,-121.01,5.00,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@|\x01\x00\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xfe',"[2453403.5, 10, 29, 5, 1, 1970, 3]"
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,13,AAC0,PIGEON,None,None,None,None,None,2004,"2,453,137.50",133,0845,1.00,Lightning,"2,453,137.50",133.00,1530,0.25,A,38.93,-120.40,5.00,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@|\x01\x00\x00\x00T\xb6\xeej\xe2\x19^\xc0\x90\xc6U]nwC@\xfe',"[2453137.5, 10, 29, 5, 1, 1970, 3]"
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,021,A32W,SLACK,None,None,None,None,None,2004,"2,453,156.50",152,1921,5.00,Debris Burning,"2,453,156.50",152.00,2024,0.10,A,38.98,-120.74,13.00,STATE OR PRIVATE,CA,17,017,El Dorado,"b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@|\x01\x00\x00\x00\xd0\xa5\xa0W\x13/^\xc0P\xbbf,\xf9}C@\xfe'","[2453156.5, 10, 29, 5, 1, 1970, 3]"
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,6,None,DEER,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1400,0.10,A,38.56,-119.91,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00\x94\xac\xa3\rt\xfa]\xc0\xe8T\x00\xc6\x92GC@\xfe',"[2453184.5, 10, 29, 5, 1, 1970, 3]"
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,7,None,STEVENOT,None,None,None,None,None,2004,"2,453,184.50",180,1600,1.00,Lightning,"2,453,189.50",185.00,1200,0.10,A,38.56,-119.93,5.00,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@|\x01\x00\x00\x00@\xe3\xaa.\xb7\xfb]\xc0\xe8T\x00\xc6\x92GC@\xfe',"[2453184.5, 10, 29, 5, 1, 1970, 3]"


In [ ]:
f_df.tail(10)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,date
990,991,1003,FS-1421048,FED,FS-FIRESTAT,FS,USLAKIF,Kisatchie National Forest,0806,Kisatchie National Forest,28,D2/19,BE1K,028 IUA 10,None,None,None,None,None,2005,"2,453,536.50",166,1405,2.00,Equipment Use,"2,453,536.50",166.00,1615,1.00,B,31.02,-93.14,5.00,USFS,LA,None,None,None,b'\x00\x01\xad\x10\x00\x00P\xa5\x14\x11\x11IW\xc0`\x04GUU\x05?@P\xa5\x14\x11\x11IW\xc0`\x04GUU\x05?@|\x01\x00\x00\x00P\xa5\x14\x11\x11IW\xc0`\x04GUU\x05?@\xfe',"[2453536.5, 10, 29, 5, 1, 1970, 3]"
991,992,1004,FS-1421052,FED,FS-FIRESTAT,FS,USLAKIF,Kisatchie National Forest,0806,Kisatchie National Forest,29,D2/20,BE1K,029 RED OAK,None,None,None,None,None,2005,"2,453,543.50",173,0800,7.00,Arson,"2,453,543.50",173.00,1130,10.00,C,31.03,-92.90,5.00,USFS,LA,None,None,None,"b'\x00\x01\xad\x10\x00\x00lQ,\xd8\x829W\xc0\x80?:\xf1\xac\x08?@lQ,\xd8\x829W\xc0\x80?:\xf1\xac\x08?@|\x01\x00\x00\x00lQ,\xd8\x829W\xc0\x80?:\xf1\xac\x08?@\xfe'","[2453543.5, 10, 29, 5, 1, 1970, 3]"
992,993,1005,FS-1421053,FED,FS-FIRESTAT,FS,USMTCGF,Custer Gallatin National Forest,0108,Custer National Forest,4,02-05-03,BU65,WAPEL DITCH FIRE,None,None,None,None,None,2005,"2,453,541.50",171,1315,5.00,Debris Burning,"2,453,541.50",171.00,1430,0.80,B,45.16,-109.27,5.00,USFS,MT,9,009,Carbon,b'\x00\x01\xad\x10\x00\x00<\t\xbf\x16lQ[\xc0\xc8\x1b=DD\x94F@<\t\xbf\x16lQ[\xc0\xc8\x1b=DD\x94F@|\x01\x00\x00\x00<\t\xbf\x16lQ[\xc0\xc8\x1b=DD\x94F@\xfe',"[2453541.5, 10, 29, 5, 1, 1970, 3]"
993,994,1006,FS-1421054,FED,FS-FIRESTAT,FS,USIDIPF,Idaho Panhandle National Forest,0104,Idaho Panhandle National Forest,44,7009,BEV6,MISSION MOUNTAIN,None,None,None,None,None,2005,"2,453,596.50",226,1545,1.00,Lightning,"2,453,596.50",226.00,2200,0.70,B,48.99,-116.30,5.00,USFS,ID,21,021,Boundary,b'\x00\x01\xad\x10\x00\x00\\{\xa0\xf4I\x13]\xc0(\x85\xebQ\xb8~H@\\{\xa0\xf4I\x13]\xc0(\x85\xebQ\xb8~H@|\x01\x00\x00\x00\\{\xa0\xf4I\x13]\xc0(\x85\xebQ\xb8~H@\xfe',"[2453596.5, 10, 29, 5, 1, 1970, 3]"
994,995,1007,FS-1421055,FED,FS-FIRESTAT,FS,USNMCIF,Cibola National Forest,0303,Cibola National Forest,110,023,BEX8,POTHOLE,None,None,None,None,None,2005,"2,453,574.50",204,2000,1.00,Lightning,"2,453,576.50",206.00,1800,0.10,A,33.72,-107.46,5.00,USFS,NM,53,053,Socorro,b'\x00\x01\xad\x10\x00\x00lP\x14K~\xddZ\xc0\xb8l\xdd\xd5\xc4\xdb@@lP\x14K~\xddZ\xc0\xb8l\xdd\xd5\xc4\xdb@@|\x01\x00\x00\x00lP\x14K~\xddZ\xc0\xb8l\xdd\xd5\xc4\xdb@@\xfe',"[2453574.5, 10, 29, 5, 1, 1970, 3]"
995,996,1008,FS-1421066,FED,FS-FIRESTAT,FS,USIDIPF,Idaho Panhandle National Forest,0104,Idaho Panhandle National Forest,35,7008,BEV6,BOULDER CREEK,None,None,None,None,None,2005,"2,453,593.50",223,1425,1.00,Lightning,"2,453,594.50",224.00,1230,0.10,A,48.59,-116.19,5.00,USFS,ID,21,021,Boundary,b'\x00\x01\xad\x10\x00\x00(I\x85N\x1b\x0c]\xc0\x10\xfd\xb7X\xf2KH@(I\x85N\x1b\x0c]\xc0\x10\xfd\xb7X\xf2KH@|\x01\x00\x00\x00(I\x85N\x1b\x0c]\xc0\x10\xfd\xb7X\xf2KH@\xfe',"[2453593.5, 10, 29, 5, 1, 1970, 3]"
996,997,1009,FS-1421074,FED,FS-FIRESTAT,FS,USMTFNF,Flathead National Forest,0110,Flathead National Forest,2,001,BL25,ASHLEY MEADOW,None,None,None,None,None,2005,"2,453,440.50",70,1530,4.00,Campfire,"2,453,440.50",70.00,1600,0.10,A,48.21,-114.62,5.00,USFS,MT,29,029,Flathead,b'\x00\x01\xad\x10\x00\x00\x88\xc4%Ce\xa7\\\xc0\x90\xc5=\xd0i\x1bH@\x88\xc4%Ce\xa7\\\xc0\x90\xc5=\xd0i\x1bH@|\x01\x00\x00\x00\x88\xc4%Ce\xa7\\\xc0\x90\xc5=\xd0i\x1bH@\xfe',"[2453440.5, 10, 29, 5, 1, 1970, 3]"
997,998,1010,FS-1421075,FED,FS-FIRESTAT,FS,USMTFNF,Flathead National Forest,0

# Cause code

In [ ]:
f_df['STAT_CAUSE_CODE'].value_counts()

1.00    495
4.00    196
9.00    106
5.00     68
7.00     64
2.00     39
3.00     17
8.00      9
6.00      6
Name: STAT_CAUSE_CODE, dtype: int64

In [ ]:
f_df['FIRE_SIZE'].value_counts().head()

0.10    565
0.25     61
0.50     47
0.30     38
1.00     33
Name: FIRE_SIZE, dtype: int64

In [ ]:
f_df['FIRE_SIZE_CLASS'].value_counts()

A    665
B    267
C     50
D      6
E      5
F      4
G      3
Name: FIRE_SIZE_CLASS, dtype: int64

In [ ]:
f_df['STAT_CAUSE_DESCR'].value_counts()

Lightning         495
Campfire          196
Miscellaneous     106
Debris Burning     68
Arson              64
Equipment Use      39
Smoking            17
Children            9
Railroad            6
Name: STAT_CAUSE_DESCR, dtype: int64

In [ ]:
f_df['COUNTY'].value_counts()

53     119
5      107
17      75
39      41
3       41
33      38
23      33
21      32
1       29
37      25
27      22
51      22
43      20
13      19
47      19
7       17
61      17
11      16
29      15
9       14
103     13
49      12
63      10
25       8
19       8
31       8
137      8
75       5
35       5
510      4
15       3
113      2
6        2
71       2
69       2
57       2
89       2
81       1
91       1
129      1
87       1
65       1
95       1
Name: COUNTY, dtype: int64

In [ ]:
DATE_COLUMN = 'date/time'
DATA_URL = ('/Users/dit/Documents/canonical/uber-raw-data-sep14.csv.gz')
            
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

data = load_data(10000)
data.head()

,date/time,lat,lon,base
0,2014-09-01 00:01:00,40.22,-74.00,B02512
1,2014-09-01 00:01:00,40.75,-74.00,B02512
2,2014-09-01 00:03:00,40.76,-73.99,B02512
3,2014-09-01 00:06:00,40.74,-73.99,B02512
4,2014-09-01 00:11:00,40.81,-73.94,B02512


In [ ]:
df = px.data.gapminder().query("year == 2007")

In [ ]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.83,31889923,974.58,AFG,4
23,Albania,Europe,2007,76.42,3600523,"5,937.03",ALB,8
35,Algeria,Africa,2007,72.30,33333216,"6,223.37",DZA,12
47,Angola,Africa,2007,42.73,12420476,"4,797.23",AGO,24
59,Argentina,Americas,2007,75.32,40301927,"12,779.38",ARG,32
